### Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn

from sklearn.preprocessing import LabelEncoder

# Feature selection
from sklearn.feature_selection import RFE

#Model
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier



# Model assessment
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


### Loading data

In [ ]:
attrition = pd.read_csv('attrition.csv')
df1 = pd.read_csv('df1.csv')
df2 = pd.read_csv('df2.csv')
target = pd.read_csv('test_target.csv')

### Data check

In [ ]:
# Checking the basic information for "attition"
attrition.info()

In [ ]:
# Checking the basic information for "target"
target.info()

In [ ]:
# Checking the basic information for "df1"
df1.info()

In [ ]:
# Checking the basic information for "df2"
df2.info()

In [ ]:
# Checking if data in "df2" contains null value
df2.isnull().sum()

In [ ]:
# Checking basic informations for data in "df2"
df2.info()

Conclusion: it can be observed that column "Attirtion" in dataframe "df2" contains (4449-4302=147) nulls --> the same numbers of records are in dataframe "target", which means that the "target" frame may have been separated from the "df2" frame

### EDA

1. New df creation by merging df1 and df2

In [ ]:
# Cration of new dataframe "data" by merging files: df1 and df2
data = df1.merge(df2)

In [ ]:
# Checking basic informations for data in "data"
data.info()

2. Verification of dataset basic information

In [ ]:
data.describe()

In [ ]:
# CSV file as archiwum, csv file consist of merged data from df1 and df2
data.to_csv('/Users/Agnieszka/.git/Klasyfikacja/data.csv', index=True)
data.to_csv('data.csv', index=True)

In [ ]:
#training dataframe creation without data for test X_test
# if target dataframe contains test data, and if "data" dataframe has nulls the same as in target this means that "train" data
# is "data"-"nulls"
train_DF = data.loc[data['Attrition'].notnull()]

# and "test" data is "data" only with "null" attition
X_test = data.loc[data['Attrition'].isnull()]

In [ ]:
train_DF

In [ ]:
X_test

In [ ]:
train_DF.describe()

In [ ]:
X_test.describe()

In [ ]:
train_DF.info()

In [ ]:
X_test.info()

In [ ]:
# column names
columns=train_DF.columns
columns

In [ ]:
len(columns)
#therefore, number of features = 36-1=35

In [ ]:
train_DF.isna().sum()

In [ ]:
print(train_DF[train_DF['NumCompaniesWorked'] == 0])

In [ ]:
#checking value in "YearsAtCompany" column
train_DF['YearsAtCompany'].describe()

In [ ]:
#checking data suspicious to be outliers in column "YearsAtCompany"
print(train_DF[train_DF['YearsAtCompany'] > 30])

3. Dupliacates

In [ ]:
#dupliacted rows

duplicates_mask=train_DF.duplicated(subset=['Age', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18',
       'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'Attrition', 'YearlyIncome'])

duplicated_train_DF = train_DF[duplicates_mask]
duplicated_train_DF
duplicated_train_DF.to_csv('Duplicates.csv')
duplicated_train_DF.to_csv('/Users/Agnieszka/.git/Klasyfikacja/Duplicates.csv')



In [ ]:
#duplicates removing
train_DF.drop_duplicates (subset=train_DF[['Age', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18',
       'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'Attrition', 'YearlyIncome']], keep='first', inplace=True)

In [ ]:
#verification of train_DF dataframe after dropping duplicates (1237 duplicates removed)
train_DF.shape

4. Unique values

In [ ]:
#Printing all unique values from individual columns
for column in train_DF.columns:

    unique_values = train_DF[column].unique()

    print(f"Unique value for column: {column}:")

    print(unique_values)

    print()

5. Outliers

Outliers verification through boxplots (rejecting columns without numerical data) - for train data

In [ ]:
for column in train_DF.columns:
    if(np.issubdtype(train_DF[column].dtype, np.number)):
        plt.figure()
        plt.boxplot(train_DF[column])
        plt.title(column)
        plt.xlabel('Column')
        plt.ylabel('Value')

Outliers verification for X_test data

In [ ]:
for column in X_test.columns:
    if(np.issubdtype(X_test[column].dtype, np.number)):
        plt.figure()
        plt.boxplot(X_test[column])
        plt.title(column)
        plt.xlabel('Column')
        plt.ylabel('Value')
plt.show()

for X_test all outliers seems to be realiable




Boxplot analysis was performed to identify outliers in the sets
Improbable values identified for columns: 'Age', 'DistanceFromHome', 'TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager'
Due to the presence of management group in the dataset, values considered as outliers for the columns 'MonthlyIncome' and 'YearlyIncome' were considered probable and left untransformed.
Columns 'EmployeeCount', 'Over18' and 'StandardHours' will be removed due to poor usability for the model (same values).
Unique value for column: Over18: ['Y']
Unique value for column: StandardHours:[80.]
Unique value for column: EmployeeCount:[1.]

Checking the impact of outliers on the change of the median value of the entire set on the example of the 'Age' column

In [ ]:
# creation of outlieres variable column

outliers = train_DF[['Age', 'DistanceFromHome', 'TotalWorkingYears', 'YearsAtCompany', 
                                 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']]

In [ ]:
#function for median calculation
def calculate_Median (data, column_name):
    median=np.median(data)
    print(f'{column_name}, median value: {median}')

In [ ]:
#calculation of median for each column in outliers
for column in outliers.columns:
    calculate_Median(outliers[column], column)
    


In [ ]:
# verification of value in column ['Age']
train_DF['Age'].describe()

In [ ]:
# creation of df with realistic age (realistic age assumed nas <=60)
df_with_real_age = train_DF.loc[train_DF['Age'] <= 60]

In [ ]:
#verification of median for realistic data (Age<=60)
df_with_real_age['Age'].describe()

Conclusion: the value of the median after rejecting outliers did not change significantly in relation to the median of the entire set. The median of the entire set = 36. (median for dataset with realistic age is 35) Since the percentage share of outliers in each of the columns listed in the 'outliers' frame is similar, the value of the median will also be assumed for the other columns. Due to the possibility of a logical error ('YearsAtCompany', 'YearsInCurrentRole') greater than 'Age' after replacing the outliers with the median, in the 'Age' column for outliers, the value 60 will be assumed as the upper, real limit.

In [ ]:
# changing "outliers" in column ['Age'] by interpolation method; 
# interpolation will be the best because almost of all data for outliers are the same value as the nearest one with the same JobRole
# Age column is treated separately because value of outliers differs from another outliers

# train_DF.loc[train_DF['Age'] > 60, 'Age'] = train_DF.loc[train_DF['Age'] > 60, 'Age'].interpolate()
# train_DF['Age'].describe()

In [ ]:
# creation of outlieres variable vithout "Age" column

# outliers_without_age = train_DF[['DistanceFromHome', 'TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole', 
#                                  'YearsSinceLastPromotion', 'YearsWithCurrManager']]

In [ ]:
# def outliers_removing (cols, dataFrame):
#     for col in cols:
#         dataFrame.loc[dataFrame[col] > 1000, col] = dataFrame.loc[dataFrame[col] > 1000, col].interpolate()

Interpolation appears to be inappropriate method due to the adjacency of the rows also containing outliers

In [ ]:
# Median value for "Age"
medianAge = np.median(train_DF['Age'])
medianAge

In [ ]:
medianAge_X_test = np.median(X_test['Age'])
medianAge_X_test

In [ ]:
# outliers removing from Age column by median method
train_DF.loc[train_DF['Age']>60, 'Age'] = medianAge

# for X_test medianAge from train_DF dataset will be used as same 
X_test.loc[X_test['Age']>60, 'Age'] = medianAge

In [ ]:
train_DF['Age'].describe()

In [ ]:
X_test['Age'].describe()

Replacing outliers for rest of columns without "Age" column

In [ ]:
outliers_without_age = train_DF[['DistanceFromHome', 'TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole', 
                                 'YearsSinceLastPromotion', 'YearsWithCurrManager']]

In [ ]:
# Function for exchanging outliers (except "Age") to median
def outliers_removing (cols, dataFrame):
    for col in cols:
        median = np.median(train_DF[col])
        dataFrame.loc[dataFrame[col] > 1000, col] = median

In [ ]:
outliers_removing(outliers_without_age, train_DF)
outliers_removing(outliers_without_age, X_test)

In [ ]:
# dataframe after outliers removing
train_DF.describe()

In [ ]:
X_test.describe()

# outliers_without_age = train_DF[['DistanceFromHome', 'TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole', 
#                                  'YearsSinceLastPromotion', 'YearsWithCurrManager']]

Taking into accout that outliers are very simillar to surroundig rows, perhaps the best way will be deletion of rows with outliers value, but first the model will be traind on outliers replaced by median

Checking of histograms after outliers removing to find if now all data are realistic

In [ ]:
train_DF.columns

In [ ]:
columns_to_plot = ['Age', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager', 'Attrition',
       'YearlyIncome']



In [ ]:
len(columns_to_plot)

In [ ]:
# histograms for train_DF

fig, axes = plt.subplots(nrows=len(columns_to_plot), figsize = (5,3*len(columns_to_plot)))

for i, column in enumerate (columns_to_plot):
    sns.histplot(data=train_DF, x=column, ax=axes[i])
    axes[i].set_title(column)
plt.tight_layout()
plt.show()

In [ ]:
# histograms for X_train

fig, axes = plt.subplots(nrows=len(columns_to_plot), figsize = (5,3*len(columns_to_plot)))

for i, column in enumerate (columns_to_plot):
    sns.histplot(data=X_test, x=column, ax=axes[i])
    axes[i].set_title(column)
plt.tight_layout()
plt.show()

6. Deletion of columns without any valuable data (one data for whole column)

In [ ]:
#X train dataframe

train_DF.drop(['EmployeeCount'], axis= 1, inplace=True)
train_DF.drop(['Over18'], axis=1, inplace=True)
train_DF.drop(['StandardHours'], axis=1, inplace=True)

# dataframe with data for testing

X_test.drop(['EmployeeCount'], axis= 1, inplace=True)
X_test.drop(['Over18'], axis=1, inplace=True)
X_test.drop(['StandardHours'], axis=1, inplace=True)

In [ ]:
train_DF.info()
# 3 column should be removed

In [ ]:
X_test.info()

### Feature Engineering

1. Uniqe value to be modified

In [ ]:
# Wypisanie wszystkich unikalnych wartości z poszczególnych kolumn
for column in train_DF.columns:

    unique_values = train_DF[column].unique()

    print(f"Unikalne wartości dla kolumny {column}:")

    print(unique_values)

    print()

2. Simple replacement

Unique values for chosen column

In [ ]:
columns_to_encode = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime', 'Attrition']

#becasue of column "Attition" in X_test has NaN, incoding is not necessary and this column will be omitted
columns_to_encode_X = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']

In [ ]:
unique_values_train_DF = train_DF[columns_to_encode].apply(lambda x: x.unique()).to_dict()
unique_values_train_DF

In [ ]:
unique_values_X_test = X_test[columns_to_encode_X].apply(lambda x: x.unique()).to_dict()
unique_values_X_test

In [ ]:
# unique_values_train_DF.to_csv('/Users/Agnieszka/.git/Klasyfikacja/uniqe_values_train_DF.csv', index=True)

In [ ]:
def encode_categorical_columns(dataframe, columns):
    for column in columns:
        unique_values = dataframe[column].unique()
        mapping = {value: i for i, value in enumerate(unique_values)}
        dataframe[column] = dataframe[column].map(mapping)
    return dataframe

In [ ]:
encoded_DF = encode_categorical_columns(train_DF, columns_to_encode)
encoded_DF

In [ ]:
encoded_DF_X = encode_categorical_columns(X_test, columns_to_encode_X)
encoded_DF_X

In [ ]:
encoded_DF_X

In [ ]:
unique_values_after_encoding = train_DF[columns_to_encode].apply(lambda x: x.unique()).to_dict()
unique_values_after_encoding

In [ ]:
unique_values_after_encoding_X = X_test[columns_to_encode].apply(lambda x: x.unique()).to_dict()
unique_values_after_encoding_X

### Data discretization

First histogram will be show for train_DF and X_train dataset to compare and control if previous data manipulation
has been done properly.

Discretization will be performed first for train_DF than for X_test

a) Age column data discretization

In [ ]:
train_DF['Age'].hist()

In [ ]:
max_Age = train_DF.loc[train_DF['Age']>50]
max_Age

In [ ]:
max_Age = train_DF.loc[train_DF['Age']>60]
max_Age

In [ ]:
# age division into bins for training and test data

train_DF['Age_Bin'] = pd.cut(train_DF.Age, labels=['17-30', '31-45', '46-60'], bins=[17, 30, 45, 60])

X_test['Age_Bin'] = pd.cut(X_test.Age, labels=['17-30', '31-45', '46-60'], bins=[17, 30, 45, 60])

In [ ]:
train_DF.head(30)

In [ ]:
#Age conversion from bins to numeric for train_DF


unique_age_bin = tuple(train_DF.Age_Bin.unique())

train_DF['Age_Bin'].replace(unique_age_bin, range(len(unique_age_bin)), inplace=True)
train_DF.drop('Age', axis=1, inplace=True)



In [ ]:
#Age conversion from bins to numeric for X_test
unique_age_bin_X = tuple(X_test.Age_Bin.unique())
X_test['Age_Bin'].replace(unique_age_bin_X, range(len(unique_age_bin_X)), inplace=True)
X_test.drop('Age', axis=1, inplace=True)


In [ ]:
train_DF['Age_Bin'].describe()

In [ ]:
X_test['Age_Bin'].describe()

b) Data discretization for rested columns

In [ ]:
train_DF['DailyRate'].hist()

In [ ]:
X_test['DailyRate'].hist()

In [ ]:
daily_rate = train_DF["DailyRate"]
categories = pd.qcut(daily_rate, q=5, labels=False)
train_DF["DailyRate"] = categories


daily_rateX = X_test["DailyRate"]
categoriesX = pd.qcut(daily_rateX, q=5, labels=False)
X_test["DailyRate"] = categoriesX

In [ ]:
train_DF['HourlyRate'].hist()

In [ ]:
X_test['HourlyRate'].hist()

In [ ]:
HourlyRate = train_DF["HourlyRate"]
categories = pd.qcut(HourlyRate, q=5, labels=False)
train_DF["HourlyRate"] = categories

HourlyRateX = X_test["HourlyRate"]
categoriesX = pd.qcut(HourlyRateX, q=5, labels=False)
X_test["HourlyRate"] = categoriesX

In [ ]:
train_DF['MonthlyIncome'].hist()

In [ ]:
X_test['MonthlyIncome'].hist()

In [ ]:
MonthlyIncome = train_DF["MonthlyIncome"]
categories = pd.qcut(MonthlyIncome, q=5, labels=False)
train_DF["MonthlyIncome"] = categories


MonthlyIncomeX = X_test["MonthlyIncome"]
categoriesX = pd.qcut(MonthlyIncomeX, q=5, labels=False)
X_test["MonthlyIncome"] = categoriesX

In [ ]:
train_DF['MonthlyRate'].hist()

In [ ]:
X_test['MonthlyRate'].hist()

In [ ]:
MonthlyRate = train_DF["MonthlyRate"]
categories = pd.qcut(MonthlyRate, q=5, labels=False)
train_DF["MonthlyRate"] = categories


MonthlyRateX = X_test["MonthlyRate"]
categoriesX = pd.qcut(MonthlyRateX, q=5, labels=False)
X_test["MonthlyRate"] = categoriesX


In [ ]:
train_DF['YearlyIncome'].hist()

In [ ]:
X_test['YearlyIncome'].hist()

In [ ]:
YearlyIncome = train_DF["YearlyIncome"]
categories = pd.qcut(YearlyIncome, q=5, labels=False)
train_DF["YearlyIncome"] = categories

YearlyIncomeX = X_test["YearlyIncome"]
categoriesX = pd.qcut(YearlyIncomeX, q=5, labels=False)
X_test["YearlyIncome"] = categoriesX

In [ ]:
train_DF.head(30)

In [ ]:
X_test.head(30)

In [ ]:
train_DF.describe()

In [ ]:
columns_to_show = list(train_DF.columns)
columns_to_show

In [ ]:
# histograms after feature engeeniring

fig, axes = plt.subplots(nrows=len(columns_to_show), figsize = (5,3*len(columns_to_show)))

for i, column in enumerate (columns_to_show):
    sns.histplot(data=train_DF, x=column, ax=axes[i])
    axes[i].set_title(column)
plt.tight_layout()
plt.show()

In [ ]:
# histograms after feature engeeniring

fig, axes = plt.subplots(nrows=len(columns_to_show), figsize = (5,3*len(columns_to_show)))

for i, column in enumerate (columns_to_show):
    sns.histplot(data=X_test, x=column, ax=axes[i])
    axes[i].set_title(column)
plt.tight_layout()
plt.show()

### Analysis of the significance of variables based on regression coefficients

Correlation verification after EDA and feature engineering

In [ ]:
plt.figure(figsize=(25,20))
sns.set_theme(style="white")
corr = train_DF.corr()
heatmap = sns.heatmap(corr, annot=True, cmap="Blues", fmt='.1g')

In [ ]:
Correlation_for_Attition=corr['Attrition']
Correlation_for_Attition

In [ ]:
# Correlation to absolute value transformation and sorting

Correlation_sorted = np.absolute(Correlation_for_Attition).sort_values()
Correlation_sorted

In [ ]:
Correlation_sorted_50perc = Correlation_sorted>0.5
Correlation_sorted_50perc

In [ ]:
significant_variable = Correlation_sorted[Correlation_sorted>0.5]
significant_variable

### X_train, y_train, X_test, y_test preparation


from train_DF following will be prepared
X_train = train_DF-train_DF['Attrition']
y_train = train_DF['Attrition']
///
from X_test following will be prepared
X_test = X_test - X_test['Attrition']
y_test = target['Attrition]

In [ ]:
# y_test preparation. y_test is the column ['Attrition] from test_target.csv file (variable "target" prepared at the begining
# the order of operations is important

In [ ]:
X_test.shape

In [ ]:
target.shape

In [ ]:
y_train = train_DF['Attrition']

In [ ]:
y_train.shape

In [ ]:
X_train = train_DF.drop('Attrition', axis=1)

In [ ]:
X_train.shape

In [ ]:
X_train.head(10)

In [ ]:
y_test = target['Attrition']

In [ ]:
y_test.shape

In [ ]:
X_test = X_test.drop('Attrition',axis=1)

In [ ]:
X_test.shape

### First simply model to check significance of variables _ ALL VARIABLES

In [ ]:
def calculate_metrics(target, prediction, average='weighted'):
    accuracy = accuracy_score(target, prediction)
    precision = precision_score(target, prediction, average=average)
    recall = recall_score(target, prediction, average=average)
    f1 = f1_score(target, prediction, average=average)
    mislabeled = (target != prediction).sum()
    total = len(target)
    return accuracy, precision, recall, f1, mislabeled, total

def print_results(metrics):
    print(f'  Accuracy:  {metrics[0]}')
    print(f'  Precision: {metrics[1]}')
    print(f'  Recall:    {metrics[2]}')
    print(f'  F1 score:  {metrics[3]}')
    print(f'  Mislabeled {metrics[4]} out of {metrics[5]}')
    print('\n')

In [ ]:
scaler = StandardScaler()

def train_model(model, grid_search_params):
  pipe = Pipeline(steps=[("scaler", scaler), ("model", model)])
  grid_pipeline = GridSearchCV(pipe, grid_search_params)
  # fit
  grid_pipeline.fit(X_train, y_train)
  print("Best CV accuracy train", grid_pipeline.best_score_)
  print("Best CV params", grid_pipeline.best_params_)
  
  pred = grid_pipeline.predict(X_test)
  metrics = calculate_metrics(y_test, pred)
  print_results(metrics)
  return grid_pipeline

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree_grid_search_params = {}

grid_search_decision_tree = train_model(decision_tree, decision_tree_grid_search_params)

In [ ]:
importances = grid_search_decision_tree.best_estimator_["model"].feature_importances_

In [ ]:
# Creating importances_df dataframe
importances_df = pd.DataFrame({"feature_names" : X_train.columns.tolist(), 
                               "importances" : importances})
                             
# Plotting bar chart, g is from graph
fig, ax = plt.subplots(figsize=(25, 10))
g = sns.barplot(x=importances_df["feature_names"], 
                y=importances_df["importances"],
                ax=ax
)
g.set_title("Feature importances", fontsize=14)

In [ ]:
# Creating importances_df dataframe
importances_df = pd.DataFrame({"feature_names" : X_train.columns.tolist(), 
                               "importances" : importances})
                             
# Plotting bar chart, g is from graph
fig, ax = plt.subplots(figsize=(25, 10))
g = sns.barplot(x=importances_df["feature_names"], 
                y=importances_df["importances"],
                ax=ax
)
g.set_title("Feature importances", fontsize=14)

### Recursive Feature Elimination for Decision Tree Classifier

Decision Tree Classifier for all variables

In [484]:
model_DTC = DecisionTreeClassifier()

In [485]:
model_DTC.fit(X_train, y_train)

DecisionTreeClassifier()

In [486]:
y_predict_train= model_DTC.predict(X_train)
y_predict_test = model_DTC.predict(X_test)

In [487]:
def calculate_metrics(target, prediction, average='weighted'):
    accuracy = accuracy_score(target, prediction)
    precision = precision_score(target, prediction, average=average)
    recall = recall_score(target, prediction, average=average)
    f1 = f1_score(target, prediction, average=average)
    mislabeled = (target != prediction).sum()
    total = len(target)
    return accuracy, precision, recall, f1, mislabeled, total

In [488]:
def print_results(metrics, classifier_id='classifier'):
    print(f'Results for {classifier_id}')
    print('----')
    print(f'  Accuracy:  {metrics[0]}')
    print(f'  Precision: {metrics[1]}')
    print(f'  Recall:    {metrics[2]}')
    print(f'  F1 score:  {metrics[3]}')
    print(f'  Mislabeled {metrics[4]} out of {metrics[5]}')
    print('\n')

In [489]:
#Metrics for train data
print_results(calculate_metrics(y_train, y_predict_train), 'Decision_Tree_all_variables_train')

Results for Decision_Tree_all_variables_train
----
  Accuracy:  1.0
  Precision: 1.0
  Recall:    1.0
  F1 score:  1.0
  Mislabeled 0 out of 3065




In [490]:
#Metrics for test data
print_results(calculate_metrics(y_test, y_predict_test), 'Decision_Tree_all_variables_test')

Results for Decision_Tree_all_variables_test
----
  Accuracy:  0.3741496598639456
  Precision: 0.7080383173677635
  Recall:    0.3741496598639456
  F1 score:  0.42932613380846046
  Mislabeled 92 out of 147




Recursive Feature Elimination 

In [491]:
model_RFE = RFE(model_DTC, n_features_to_select = 7)

In [492]:
RFE_fit = model_RFE.fit(X_train, y_train)

In [493]:
RFE_fit.support_

array([ True, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True, False, False, False,  True, False, False, False,
        True, False,  True, False, False])

In [494]:
significant_variables_acc_RFE = X_train.columns[RFE_fit.support_]
significant_variables_acc_RFE

Index(['EmployeeNumber', 'DistanceFromHome', 'NumCompaniesWorked',
       'PercentSalaryHike', 'TotalWorkingYears', 'YearsInCurrentRole',
       'YearsWithCurrManager'],
      dtype='object')

Decision Tree Classifier for RFE significant variables

In [495]:
selected_columns = ['EmployeeNumber', 'DistanceFromHome', 'JobRole', 'NumCompaniesWorked',
       'PercentSalaryHike', 'TotalWorkingYears', 'YearsAtCompany']

In [496]:
X_train_RFE = X_train[selected_columns]
X_test_RFE = X_test[selected_columns]

In [497]:
model_DTC_RFE_variables = DecisionTreeClassifier()

In [498]:
model_DTC_RFE_variables.fit(X_train_RFE, y_train)

DecisionTreeClassifier()

In [499]:
y_predict_RFE_variables_train = model_DTC_RFE_variables.predict(X_train_RFE)

In [500]:
y_predict_RFE_variables_test = model_DTC_RFE_variables.predict(X_test_RFE)

In [501]:
print_results(calculate_metrics(y_train, y_predict_RFE_variables_train), 'Decision_Tree_RFE_Significant_variables_train')

Results for Decision_Tree_RFE_Significant_variables_train
----
  Accuracy:  1.0
  Precision: 1.0
  Recall:    1.0
  F1 score:  1.0
  Mislabeled 0 out of 3065




In [502]:
print_results(calculate_metrics(y_test, y_predict_RFE_variables_test), 'Decision_Tree_RFE_Significant_variables_test')

Results for Decision_Tree_RFE_Significant_variables_test
----
  Accuracy:  0.3741496598639456
  Precision: 0.800060171488743
  Recall:    0.3741496598639456
  F1 score:  0.406784952787085
  Mislabeled 92 out of 147


